In [1]:
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
import random
from random import sample
from collections import Counter, defaultdict
from itertools import chain
import math
from tqdm import tqdm
tqdm.pandas()
import json

# 데이터 불러오기
- song_meta에 인기도가 필요함.
- 가수 유사도 계산을 위한 word2vec 모델 필요

In [2]:
train = pd.read_json('train.json')
song_meta = pd.read_json('song_meta_pop.json')
w2v_model = Word2Vec.load('../artist_w2v.model')

### 플레이리스트에 있는 가수 컬럼으로 만들기.

In [3]:
# 노래 id가 key, 값은 그 노래를 부른 가수 리스트
songid_to_artists = dict(zip(song_meta['id'],song_meta['artist_name_basket']))

# 각 플레이리스트에 포함된 노래를 부른 가수들을 가져오기 위한 함수
def get_artist(playlist):
    artists = set()
    for song in playlist:
        artists.update(songid_to_artists[song])
    return list(artists)

train['artists'] = train['songs'].progress_apply(get_artist)

100%|███████████████████████████████████████████████████████████████████████| 115071/115071 [00:04<00:00, 24207.80it/s]


### artist 펼치기

In [4]:
artist_flat = pd.DataFrame([[i, x] for i, y in song_meta['artist_name_basket'].iteritems() for x in y], columns=['i','artist'])
artist_flat = artist_flat.set_index('i')
artist_flat.loc[artist_flat['artist'] == "",'artist'] = 'unknown'

### 펼친 df를 dict로 만들기.

In [5]:
#  key = 가수 value = 부른 노래 리스트.
artist_song_dict = defaultdict(list)

for i, artist in tqdm(artist_flat['artist'].iteritems(), total = len(artist_flat)):
    artist_song_dict[artist].append(i)
print(len(artist_song_dict))

100%|█████████████████████████████████████████████████████████████████████| 779485/779485 [00:00<00:00, 1088628.74it/s]

111304


In [6]:
# json.dump(artist_song_dict, open('db/artist_song_dict.json', 'w'))

In [7]:
#  artist_song_dict = dict(json.loads(open('db/artist_song_dict.json', 'r').read()))

# 모델
### word2vec을 기반으로 추천 받은 가수들의 곡을 기준 가수 곡과 적절히 섞어서 추천한다.

In [8]:
class Recommender:
    
    def __init__(self):
        self.song_meta = pd.read_json('song_meta_pop.json')
        self.train = pd.read_json('train.json')
        self.all_tags_set = set(chain(*train['tags']))
        self.id_to_tag = dict(zip(range(len(self.all_tags_set)), self.all_tags_set))
        self.tag_to_id = dict(zip(self.all_tags_set, range(len(self.all_tags_set)) ))
        
        #for singer_recommend
        self.artist_w2v_model = Word2Vec.load('../artist_w2v.model')
        self.artist_song_dict = dict(json.loads(open('db/artist_song_dict.json', 'r').read())) # 위에서 만든 dict
    
    def singer_recommend(self, user, rec_songs_cnt = 30, rate_of_familiar_songs = 0.3, artist_sample = 5):
        #아는 가수의 노래 수
        song_cnt_from_fam = int(rec_songs_cnt * rate_of_familiar_songs)
        #추천 받은 가수의 노래 수
        song_cnt_from_rec = int(rec_songs_cnt * (1-rate_of_familiar_songs))
        
        artist_songs = []
        new_artist_songs = []
        final_recommendation = []
        user_recommend_artists = []
    
        #유저 아티스트 랜덤 샘플 가져오기
        if len(user['artists'].values) > artist_sample:
            user_random_artist_list = random.sample(*user['artists'].values, artist_sample)
        else:
            user_random_artist_list = list(*user['artists'].values)
        
        #유사 가수 찾기  - (가수 전체로 찾는다.)
        temp = self.artist_w2v_model.wv.most_similar(user_random_artist_list, topn = 7)
        user_recommend_artists.extend([x for x , _ in temp])
        
        # 기존 가수 노래 가져오기
        for artist_ in user_random_artist_list:
            temp_songs = self.song_meta.iloc[self.artist_song_dict[artist_]].sort_values(by= 'popularity', ascending = False)[:10]
            artist_songs.extend(temp_songs['id'])
        # 새 가수 노래 가져오기    
        for artist_ in user_recommend_artists:
            temp_songs = self.song_meta.iloc[self.artist_song_dict[artist_]].sort_values(by= 'popularity', ascending = False)[:10]
            new_artist_songs.extend(temp_songs['id'])
        # 기존 가수 곡 중 이미 있는 노래는 제거
        artist_songs =  set(artist_songs) - set(*user['songs'])
        
        #노래 샘플로 뽑기
        #뽑힌 기존 가수 노래가 설정된 기존 가수 노래 수보다 크면 수를 맞춘다.
        if len(artist_songs) >= song_cnt_from_fam:
            final_recommendation.extend(sample(artist_songs,  song_cnt_from_fam   ))
        else:
            final_recommendation.extend(artist_songs)
        
        #추천 가수 노래가 설정된 추천가수노래보다 크면 수를 맞춘다.
        if len(new_artist_songs) >= song_cnt_from_rec:
            final_recommendation.extend(sample(new_artist_songs,  song_cnt_from_rec    ))
        else:
            final_recommendation.extend(new_artist_songs)
            
        return final_recommendation

In [9]:
%%time
rec = Recommender()

Wall time: 17.9 s


In [125]:
sample_user = train.sample(1)
print('샘플 유저 : \n')
display(sample_user)

샘플 유저 : 



,tags,id,plylst_title,songs,like_cnt,updt_date,artists
23720,[발라드],79111,가을에 들을 말한 노래,"[452583, 321059, 287003, 198388, 603107, 31758...",10,2009-09-26 14:30:56.000,"[부활, 마리오, 김경호, 다이나믹 듀오, 이승철, 럼블피쉬, 케이윌, 플라이 투 ..."


### 샘플 유저로 추천 테스트 해보기

In [127]:
final_recommendation =  rec.singer_recommend(sample_user)

In [128]:
display(song_meta.iloc[final_recommendation][['song_name','artist_name_basket']])

,song_name,artist_name_basket
635537,보고 싶다,[김범수]
9033,그게 나야,[김동률]
247663,Smile Again,[KCM]
238621,Missing You,[플라이 투 더 스카이]
138931,바람이 되어서라도,[환희]
695770,300원짜리 커피,[소울스타 (SOULSTAR)]
187486,내 사랑 내 곁에,[럼블피쉬]
279076,내 생에 아름다운,[케이윌]
538124,결혼해줄래,[이승기]
444921,사랑은 아프려고 하는 거죠,[엠씨더맥스 (M.C the MAX)]
